# Twitter Streaming with pySpark

In [1]:
# import necessary packages
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

Creating spark context

In [2]:
sc = SparkContext()

In [3]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

C:\Program Files\spark-3.2.0-bin-hadoop3.2\python\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:

socket_stream = ssc.socketTextStream("127.0.0.1", 1234)

In [8]:
# lines of tweets with socket_stream window of size 60, or 60 seconds windows of time
lines = socket_stream.window(60) 

In [9]:

from collections import namedtuple

fields = ("hashtag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [10]:
# here we apply different operations on the tweets and save them to a temporary sql table

( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) 
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a dataframe
  .limit(10).registerTempTable("tweets") ) ) # Registers only top 10 hashtags to a table.

Run the python file so that it can bring in streaming data.

In [11]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
%matplotlib inline 

In [12]:
# start streaming and wait couple of minutes to get enought tweets
ssc.start()

In [13]:

count = 0
while count < 5:
    
    time.sleep(5)
    top_10_tags = sqlContext.sql( 'Select hashtag, count from tweets' )
    top_10_df = top_10_tags.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="hashtag", data=top_10_df)
    plt.show()
    count = count + 1
    print(conut)

AnalysisException: Table or view not found: tweets; line 1 pos 27;
'Project ['hashtag, 'count]
+- 'UnresolvedRelation [tweets], [], false


In [ ]:
ssc.stop()